In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
model_conv = models.alexnet(pretrained = True)

In [4]:
for param in model_conv.parameters():
    param.requires_grad = False

In [10]:
import glob
imgs_train = []
imgs_test = []
imgs_train_paths = []
labels_train = []
labels_test = []
for img in glob.glob("C:/Users/etuna/Desktop/hw3/dataset/dataset/training/*"):
    imgs_train_paths.append(img)
    n= cv2.imread(img)
    imgs_train.append(n)
    
for img in glob.glob("C:/Users/etuna/Desktop/hw3/dataset/dataset/test/*"):
    n= cv2.imread(img)
    imgs_test.append(n)
    
with open('C:/Users/etuna/Desktop/hw3/dataset/dataset/training_labels.txt', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_train = list(map(int,labelStr))
    
with open('C:/Users/etuna/Desktop/hw3/dataset/dataset/test_labels.txt', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_test = list(map(int,labelStr))
    
print("Train set size:",len(imgs_train))
print("Test set size:",len(imgs_test))
print("Train Labels size:", len(labels_train))
print("Train Labels size:", len(labels_test))
print(labels_train)

label_dict = {}
for l in labels_train:
    if not l in label_dict:
        label_dict[l] = 1
    else:
        label_dict[l] += 1
print(label_dict)

Train set size: 186
Test set size: 144
Train Labels size: 186
Train Labels size: 144
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
{1: 60, 2: 88, 3: 38}


In [11]:
minc = min(label_dict,key=label_dict.get)
label_dict[minc]

38

In [65]:
imgs_train_paths

['C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr1.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr10.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr100.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr101.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr102.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr103.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr104.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr105.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr106.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr107.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr108.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr109.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr11.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr110.jpg',
 'C:/Users/etuna/Desktop/hw3/dataset/dataset/training\\tr111.jpg',

In [64]:
imgs_train_paths[0].split('\\')[1]

'tr1.jpg'

In [8]:
print(str(len(imgs_train)))
print(label_dict)

186
{1: 60, 2: 88, 3: 38}


In [15]:
one_valid = int(label_dict[1]*0.2)
two_valid = int(label_dict[2]*0.2)
three_valid = int(label_dict[3]*0.2)
print("valids:",one_valid,two_valid,three_valid)

valids: 12 17 7


In [31]:
print(str(len(imgs_train)))
print(imgs_train_paths[1].split("\\")[1])

186
tr10.jpg


In [32]:
def processDataset():
    train_path = "C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/train/"
    valid_path = "C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/valid/"
    test_path = "C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/test/"
    one_valid = int(label_dict[1]*0.2)
    two_valid = int(label_dict[2]*0.2)
    three_valid = int(label_dict[3]*0.2)
    print("valids:",one_valid,two_valid,three_valid)
    
    t_path = "C:/Users/etuna/Desktop\hw3/dataset/dataset/dir/"
    ## save to related dirs

    for i, img in enumerate(imgs_train):
        if labels_train[i] == 1:
            if one_valid > 0:
                cv2.imwrite(valid_path+"1/"+imgs_train_paths[i].split("\\")[1], img)
                one_valid -= 1
            else:
                cv2.imwrite(train_path+"1/"+imgs_train_paths[i].split("\\")[1], img)
        elif labels_train[i] == 2:
            if two_valid > 0:
                cv2.imwrite(valid_path+"2/"+imgs_train_paths[i].split("\\")[1], img)
                two_valid -= 1
            else:
                cv2.imwrite(train_path+"2/"+imgs_train_paths[i].split("\\")[1], img)
        else:
            if three_valid > 0:
                cv2.imwrite(valid_path+"3/"+imgs_train_paths[i].split("\\")[1], img)
                three_valid -= 1
            else:
                cv2.imwrite(train_path+"3/"+imgs_train_paths[i].split("\\")[1], img)

    for i, img in enumerate(imgs_test):
        if labels_test[i] == 1:
            cv2.imwrite(test_path+"1/"+imgs_test_pathsb[i], img)
        elif labels_test[i] == 2:
            cv2.imwrite(test_path+"2/"+imgs_test_pathsb[i], img)
        else:
            cv2.imwrite(test_path+"3/"+imgs_test_pathsb[i], img)
    print("Complete.")


In [33]:
processDataset()

valids: 12 17 7
Complete.


In [79]:
def balanceSets(imgs_trainp, labels_trainp):
    train_balance_path = "C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/training_balanced/"
    valid_balance_path = "C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/valid_balanced/"
    label_dict = {}
    for l in labels_train:
        if not l in label_dict:
            label_dict[l] = 1
        else:
            label_dict[l] += 1
    print(label_dict)
    minc = min(label_dict,key=label_dict.get)
    print(str(minc)+":"+str(label_dict[minc]))
    
    balance_dict = {}
    train_labelStr= ""
    valid_labelStr= ""
    for i, img in enumerate(imgs_trainp):
        if not labels_trainp[i] in balance_dict:
            cv2.imwrite(train_balance_path+imgs_train_paths[i].split('\\')[1], img)
            train_labelStr += str(labels_trainp[i])
            balance_dict[labels_trainp[i]] = 1
        else:
            if balance_dict[labels_trainp[i]] < label_dict[minc]:
                cv2.imwrite(train_balance_path+imgs_train_paths[i].split('\\')[1], img)
                balance_dict[labels_trainp[i]] += 1
                train_labelStr += str(labels_trainp[i])
            else: # save as validation
                cv2.imwrite(valid_balance_path+imgs_train_paths[i].split('\\')[1], img)
                valid_labelStr += str(labels_trainp[i])
                continue
    with open("C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/training_balanced_labels", "w") as text_file:
        text_file.write(train_labelStr)
    with open("C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/valid_balanced_labels", "w") as text_file:
        text_file.write(valid_labelStr)
    print(balance_dict)
    print(train_labelStr)
    print(valid_labelStr)
    

In [78]:
imgs_traincopy= copy.deepcopy(imgs_train)
labels_traincopy= copy.deepcopy(labels_train)
balanceSets(imgs_traincopy, labels_traincopy)

{1: 60, 2: 88, 3: 38}
3:38
{1: 38, 2: 38, 3: 38}
111111111111111111111111111111111111112222222222222222222222222222222222222233333333333333333333333333333333333333
111111111111111111111122222222222222222222222222222222222222222222222222
111


In [80]:
print(model_conv)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [37]:
model_conv.classifier[ 6 ] = nn.Linear( in_features = 4096, out_features = 3, bias = True )
model_conv = model_conv.to(device) # use the GPU

In [38]:
criterion = nn.CrossEntropyLoss()

In [39]:
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR( optimizer_conv, step_size=30, gamma= 0.1)

In [16]:
img_train_balanced = []
img_valid_balanced = []
img_test_balanced = []
imgs_train_pathsb = []
imgs_valid_pathsb = []
imgs_test_pathsb = []

for img in glob.glob("C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/training_balanced/*"):
    n= cv2.imread(img)
    img_train_balanced.append(n)
    imgs_train_pathsb.append(img.split('\\')[1])
for img in glob.glob("C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/valid_balanced/*"):
    n= cv2.imread(img)
    imgs_valid_pathsb.append(img.split('\\')[1])
    img_valid_balanced.append(n)
for img in glob.glob("C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/test/*"):
    n= cv2.imread(img)
    img_test_balanced.append(n)
    imgs_test_pathsb.append(img.split('\\')[1])
with open('C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/training_balanced_labels', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_train_balanced = list(map(int,labelStr))
with open('C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/valid_balanced_labels', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_valid_balanced = list(map(int,labelStr))
with open('C:/Users/etuna/Desktop/hw3/dataset/dataset/balanced/test_labels.txt', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_test = list(map(int,labelStr))

print(str(len(img_train_balanced)))
print(str(len(img_valid_balanced)))
print(str(len(img_test_balanced)))
print(imgs_train_pathsb)
print("---")
print(imgs_valid_pathsb)
print("---")
print(imgs_test_pathsb)
print("---")


114
72
144
['tr1.jpg', 'tr10.jpg', 'tr100.jpg', 'tr101.jpg', 'tr102.jpg', 'tr103.jpg', 'tr104.jpg', 'tr105.jpg', 'tr106.jpg', 'tr107.jpg', 'tr108.jpg', 'tr109.jpg', 'tr11.jpg', 'tr110.jpg', 'tr111.jpg', 'tr112.jpg', 'tr113.jpg', 'tr114.jpg', 'tr115.jpg', 'tr116.jpg', 'tr117.jpg', 'tr118.jpg', 'tr119.jpg', 'tr12.jpg', 'tr120.jpg', 'tr121.jpg', 'tr122.jpg', 'tr123.jpg', 'tr124.jpg', 'tr125.jpg', 'tr126.jpg', 'tr127.jpg', 'tr128.jpg', 'tr129.jpg', 'tr13.jpg', 'tr130.jpg', 'tr131.jpg', 'tr132.jpg', 'tr153.jpg', 'tr154.jpg', 'tr155.jpg', 'tr156.jpg', 'tr157.jpg', 'tr158.jpg', 'tr159.jpg', 'tr16.jpg', 'tr160.jpg', 'tr161.jpg', 'tr162.jpg', 'tr163.jpg', 'tr164.jpg', 'tr165.jpg', 'tr166.jpg', 'tr167.jpg', 'tr168.jpg', 'tr169.jpg', 'tr17.jpg', 'tr170.jpg', 'tr171.jpg', 'tr172.jpg', 'tr173.jpg', 'tr174.jpg', 'tr175.jpg', 'tr176.jpg', 'tr177.jpg', 'tr178.jpg', 'tr179.jpg', 'tr18.jpg', 'tr180.jpg', 'tr181.jpg', 'tr182.jpg', 'tr183.jpg', 'tr184.jpg', 'tr185.jpg', 'tr186.jpg', 'tr19.jpg', 'tr65.jpg'

In [96]:
t_path = "C:/Users/etuna/Desktop\hw3/dataset/dataset/balanced_dir/"
## save to related dirs
for i, img in enumerate(img_train_balanced):
    if labels_train_balanced[i] == 1:
        cv2.imwrite(t_path+"train/1/"+imgs_train_pathsb[i], img)
    elif labels_train_balanced[i] == 2:
        cv2.imwrite(t_path+"train/2/"+imgs_train_pathsb[i], img)
    else:
        cv2.imwrite(t_path+"train/3/"+imgs_train_pathsb[i], img)

for i, img in enumerate(img_valid_balanced):
    if labels_valid_balanced[i] == 1:
        cv2.imwrite(t_path+"valid/1/"+imgs_valid_pathsb[i], img)
    elif labels_valid_balanced[i] == 2:
        cv2.imwrite(t_path+"valid/2/"+imgs_valid_pathsb[i], img)
    else:
        cv2.imwrite(t_path+"valid/3/"+imgs_valid_pathsb[i], img)
        
for i, img in enumerate(img_test_balanced):
    if labels_test[i] == 1:
        cv2.imwrite(t_path+"test/1/"+imgs_test_pathsb[i], img)
    elif labels_test[i] == 2:
        cv2.imwrite(t_path+"test/2/"+imgs_test_pathsb[i], img)
    else:
        cv2.imwrite(t_path+"test/3/"+imgs_test_pathsb[i], img)
print("Complete.")

Complete.


In [53]:
img_valid_balanced[0].shape

(256, 256, 3)

In [48]:
def train_model(image_datasets, model, b, optimizer,scheduler, num_epochs):
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 1, shuffle = True, num_workers = 4) for x in ['train', 'valid', 'test']}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_no_corrects = 0
    for epoch in range(num_epochs):
        print("epoch:"+str(epoch))
         # Set the model to the training mode for updating the weights using
         # the first portion of training images
        model.train()
        
        for inputs, labels in dataloaders['train']: # iterate over data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        # Set the model to the evaluation mode for selecting the best network
        # based on the number of correctly classified validation images
        model.eval()
        best_no_corrects_training = 0
        no_corrects_training = 0
        no_samples_training = 0
        correct_dict_training = {}
        actual_dict_training = {}
        for inputs, labels in dataloaders['train']:
            no_samples_training += 1
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                no_corrects_training += torch.sum(preds == labels.data)
                if preds == labels.data:
                    if not preds.item() in correct_dict_training:
                        correct_dict_training[labels.data.item()] = 1
                    else:
                        correct_dict_training[labels.data.item()] += 1
                if not labels.data.item() in actual_dict_training:
                    actual_dict_training[labels.data.item()] = 1
                else:
                    actual_dict_training[labels.data.item()] += 1
        if no_corrects_training > best_no_corrects_training:
            best_no_corrects_training = no_corrects_training
        
        no_corrects = 0
        no_samples = 0
        correct_dict = {}
        actual_dict = {}
        for inputs, labels in dataloaders['valid']:
            no_samples += 1
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                no_corrects += torch.sum(preds == labels.data)
                if preds == labels.data:
                    if not preds.item() in correct_dict:
                        correct_dict[labels.data.item()] = 1
                    else:
                        correct_dict[labels.data.item()] += 1
                if not labels.data.item() in actual_dict:
                    actual_dict[labels.data.item()] = 1
                else:
                    actual_dict[labels.data.item()] += 1
        if no_corrects > best_no_corrects:
            best_no_corrects = no_corrects
            best_model_wts = copy.deepcopy(model.state_dict())

        best_no_corrects_test = 0
        no_corrects_test = 0
        no_samples_test = 0
        correct_dict_test = {}
        actual_dict_test = {}
        for inputs, labels in dataloaders['test']:
            no_samples_test += 1
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                no_corrects_test += torch.sum(preds == labels.data)
                if preds == labels.data:
                    if not preds.item() in correct_dict_test:
                        correct_dict_test[labels.data.item()] = 1
                    else:
                        correct_dict_test[labels.data.item()] += 1
                if not labels.data.item() in actual_dict_test:
                    actual_dict_test[labels.data.item()] = 1
                else:
                    actual_dict_test[labels.data.item()] += 1
        if no_corrects_test > best_no_corrects_test:
            best_no_corrects_test = no_corrects_test
            #best_model_wts = copy.deepcopy(model.state_dict())
        scheduler.step()
     # Load the weights of the best network
    model.load_state_dict(best_model_wts)
    print("Training -----------------------------------")
    print("Best no correct:"+str(best_no_corrects_training.item()))   
    print("Number of samples(valid):"+str(no_samples_training))
    print("Accuracy:"+str(best_no_corrects_training / no_samples_training))
    print("Accuracy on class 1:"+str(correct_dict_training.get(1, 0)/actual_dict_training.get(1, 1)))
    print("Accuracy on class 2:"+str(correct_dict_training.get(2, 0)/actual_dict_training.get(2, 1)))
    print("Accuracy on class 3:"+str(correct_dict_training.get(3, 0)/actual_dict_training.get(3, 1)))
    print("-------------")
    print("Validation -----------------------------------")
    print("Best no correct:"+str(best_no_corrects.item()))   
    print("Number of samples(valid):"+str(no_samples))
    print("Accuracy:"+str(best_no_corrects / no_samples))
    print("Accuracy on class 1:"+str(correct_dict.get(1, 0)/actual_dict.get(1, 1)))
    print("Accuracy on class 2:"+str(correct_dict.get(2, 0)/actual_dict.get(2, 1)))
    print("Accuracy on class 3:"+str(correct_dict.get(3, 0)/actual_dict.get(3, 1)))
    print("-------------")
    print("Test : ")
    print("Best no correct:"+str(best_no_corrects_test.item()))   
    print("Number of samples(valid):"+str(no_samples_test))
    print("Accuracy:"+str(best_no_corrects_test / no_samples_test))
    print("Accuracy on class 1:"+str(correct_dict_test.get(1, 0)/actual_dict_test.get(1, 1)))
    print("Accuracy on class 2:"+str(correct_dict_test.get(2, 0)/actual_dict_test.get(2, 1)))
    print("Accuracy on class 3:"+str(correct_dict_test.get(3, 0)/actual_dict_test.get(3, 1)))
    return model

In [50]:
data_dir = 'C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/'
data_transforms = {
 'train': transforms.Compose([
 # put the input to Tensor format in order to use torch
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
     ]),
 'valid': transforms.Compose([
 # put the input to Tensor format in order to use torch
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
     ]),
 'test': transforms.Compose([
 # put the input to Tensor format in order to use torch
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
     ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
 data_transforms[x]) for x in ['train', 'valid', 'test']}
print(image_datasets)

{'train': Dataset ImageFolder
    Number of datapoints: 150
    Root location: C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ), 'valid': Dataset ImageFolder
    Number of datapoints: 36
    Root location: C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/valid
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ), 'test': Dataset ImageFolder
    Number of datapoints: 144
    Root location: C:/Users/etuna/Desktop/hw3/dataset/dataset/dir/test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )}


In [52]:
model = train_model(image_datasets=image_datasets,model= model_conv, b = 0, optimizer=optimizer_conv,scheduler=exp_lr_scheduler, num_epochs=30 )

epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
Training -----------------------------------
Best no correct:149
Number of samples(valid):150
Accuracy:tensor(0.9933)
Accuracy on class 1:1.0
Accuracy on class 2:1.0
Accuracy on class 3:0.0
-------------
Validation -----------------------------------
Best no correct:28
Number of samples(valid):36
Accuracy:tensor(0.7778)
Accuracy on class 1:1.0
Accuracy on class 2:0.0
Accuracy on class 3:0.0
-------------
Test : 
Best no correct:64
Number of samples(valid):144
Accuracy:tensor(0.4444)
Accuracy on class 1:0.7719298245614035
Accuracy on class 2:0.2564102564102564
Accuracy on class 3:0.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 